# Coursera Applied Data Science Capstone Week 3
This is the Notebook created for the course Applied Data Science

In [ ]:
# Install packages
!pip install beautifulsoup4 -q
!pip install lxml -q
!pip install geocoder

In [176]:
import numpy
import pandas
import requests
from bs4 import BeautifulSoup
import geocoder

In [177]:
# Download the HTML page
!wget -q -O 'wikipedia_canada_postal_codes.html' 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
!wget -q -O 'canada_geodata.csv' 'https://cocl.us/Geospatial_data'

In [178]:
canada_geodata = pandas.read_csv('canada_geodata.csv')

In [179]:
# Define the function that will be used on groupby
def joinstr(x):
    return pandas.Series(dict(Neighbourhood = "%s" % ', '.join(x['Neighbourhood'])))

In [186]:
# DataFrame to hold the data columns
postal_codes = pandas.DataFrame(columns=['PostalCode', 'Borough', 'Neighbourhood'])

with open('wikipedia_canada_postal_codes.html') as html_file:
    # Load html
    soap = BeautifulSoup(html_file)
    
    # select the table
    postalcode_table = soap.table
    
    # create iterator
    rows = iter(postalcode_table.find_all('tr'))
    
    # skip the table head
    next(rows)
    
    # foreach row
    for postal_row in rows:
        # select each col
        postalcode = postal_row.contents[1].text.strip()
        borough = postal_row.contents[3].text.strip()
        neighbourhood = postal_row.contents[5].text.strip()
        
        # if not assigned borough: skip it
        if 'assigned' in borough:
            continue
        
        # if not assigned neighbourhood: copy the borough
        if  'assigned' in neighbourhood:
            neighbourhood = borough
        
        # append the row into the data frame
        postal_codes = postal_codes.append({  'PostalCode': postalcode, 'Borough': borough, 'Neighbourhood': neighbourhood}, ignore_index=True)

postal_codes = postal_codes.groupby(['PostalCode', 'Borough']).apply(joinstr).reset_index()
postal_codes.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [187]:
postal_codes.shape

(103, 3)

In [188]:
def get_lat(postal_code):
    return canada_geodata[canada_geodata['Postal Code'] == postal_code].iat[0, 1]

def get_lng(postal_code):
    return canada_geodata[canada_geodata['Postal Code'] == postal_code].iat[0, 2]

In [192]:
# Create the Latitude and Longitude columns based on the Postal Code
postal_codes['Latitude'] = postal_codes['PostalCode'].apply(get_lat)
postal_codes['Longitude'] = postal_codes['PostalCode'].apply(get_lng)
postal_codes.head(50)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
